In [ ]:
#!/usr/bin/env python3
"""
Analyze existing reports to find all section names that start with "##"
"""

import sqlite3
import re
from collections import Counter
from pathlib import Path

def analyze_report_sections(db_path: str):
    """Analyze all reports to find unique section names"""
    
    conn = sqlite3.connect(db_path)
    
    # Get all report content
    cursor = conn.execute("SELECT report_id, content FROM reports")
    
    section_counter = Counter()
    total_reports = 0
    reports_with_sections = 0
    
    # Pattern to match sections starting with "##"
    section_pattern = r'^##\s*(.+?)(?:$|\n)'
    
    print("Analyzing report sections...")
    
    for report_id, content in cursor:
        total_reports += 1
        
        if not content:
            continue
            
        # Find all sections in this report
        section_matches = re.findall(section_pattern, content, re.MULTILINE)
        
        if section_matches:
            reports_with_sections += 1
            
        # Process and count sections
        for section_line in section_matches:
            section_line = section_line.strip()
            
            # Check if there's a colon
            if ':' in section_line:
                # Use everything before the colon
                section_name = section_line.split(':', 1)[0].strip()
            else:
                # Use the first word
                section_name = section_line.split()[0] if section_line.split() else section_line
            
            if section_name:
                section_counter[section_name] += 1
    
    conn.close()
    
    print(f"\n=== SECTION ANALYSIS RESULTS ===")
    print(f"Total reports: {total_reports}")
    print(f"Reports with sections: {reports_with_sections}")
    print(f"Unique section names: {len(section_counter)}")
    
    print(f"\n=== ALL SECTION NAMES (sorted by frequency) ===")
    for section_name, count in section_counter.most_common():
        print(f"{count:6d} | {section_name}")
    
    return section_counter



analyze_report_sections("base_database.sqlite")

In [ ]:
#!/usr/bin/env python3
"""
Analyze existing reports to find all section names and their content lengths
"""

import sqlite3
import re
from collections import Counter
from pathlib import Path
import statistics

def analyze_report_sections(db_path: str):
    """Analyze all reports to find unique section names and content lengths"""
    
    conn = sqlite3.connect(db_path)
    
    # Get all report content
    cursor = conn.execute("SELECT report_id, content FROM reports")
    
    section_counter = Counter()
    section_lengths = []
    section_length_by_name = {}
    total_reports = 0
    reports_with_sections = 0
    
    print("Analyzing report sections and their lengths...")
    
    for report_id, content in cursor:
        total_reports += 1
        
        if not content:
            continue
            
        # Split content by section headers and process each section
        sections = extract_sections(content)
        
        if sections:
            reports_with_sections += 1
            
        for section_name, section_content in sections:
            section_counter[section_name] += 1
            content_length = len(section_content.strip())
            section_lengths.append(content_length)
            
            if section_name not in section_length_by_name:
                section_length_by_name[section_name] = []
            section_length_by_name[section_name].append(content_length)
    
    conn.close()
    
    print(f"\n=== SECTION ANALYSIS RESULTS ===")
    print(f"Total reports: {total_reports}")
    print(f"Reports with sections: {reports_with_sections}")
    print(f"Unique section names: {len(section_counter)}")
    print(f"Total sections found: {len(section_lengths)}")
    
    if section_lengths:
        print(f"\n=== SECTION LENGTH STATISTICS ===")
        print(f"Mean section length: {statistics.mean(section_lengths):.1f} characters")
        print(f"Median section length: {statistics.median(section_lengths):.1f} characters")
        print(f"Max section length: {max(section_lengths):,} characters")
        print(f"Min section length: {min(section_lengths):,} characters")
        print(f"Standard deviation: {statistics.stdev(section_lengths):.1f} characters")
    
    print(f"\n=== SECTION NAMES WITH LENGTH STATS (sorted by frequency) ===")
    for section_name, count in section_counter.most_common():
        lengths = section_length_by_name[section_name]
        mean_len = statistics.mean(lengths)
        max_len = max(lengths)
        print(f"{count:6d} | {mean_len:7.1f} avg | {max_len:7,} max | {section_name}")
    
    return section_counter, section_lengths

def extract_sections(content: str):
    """Extract sections from report content"""
    sections = []
    
    # Split by section headers (##)
    parts = re.split(r'^(##\s*.+?)$', content, flags=re.MULTILINE)
    
    current_section_name = None
    
    for i, part in enumerate(parts):
        part = part.strip()
        if not part:
            continue
            
        # Check if this part is a section header
        if part.startswith('##'):
            # Extract section name
            section_line = part[2:].strip()  # Remove ##
            
            if ':' in section_line:
                section_name = section_line.split(':', 1)[0].strip()
            else:
                section_name = section_line.split()[0] if section_line.split() else section_line
            
            current_section_name = section_name
            
        elif current_section_name:
            # This is section content
            sections.append((current_section_name, part))
            current_section_name = None  # Reset for next section
    
    return sections


analyze_report_sections("base_database.sqlite")